In [2]:
import numpy as np
import matplotlib.pyplot as plt
from helpers import folding, unfold, percentage_difference

# Systematic Variations and Unfolding

In this notebook, the effect of systematic variations on reco or unfolded distributions is investigated. We will work through a simple 2 bin example to understand the following:
  * The effect of a detector uncertaintiy 
  * The effect of a detector uncertainity which only differs from the nominal by a different acceptance and efficiency - i.e. the migration matrix is the same
  * The effect of a signal modelling uncertainity
    * evaluating the uncertainity with two mehtods

The unfolding will be simple matrix inversion. 

#### Detector uncertainty
First we set up a nominal sample from a truth, $t_\text{nominal}$. Then define a migration matrix, $M_\text{nominal}$, acceptance $a_\text{nominal}$ and efficiency $\epsilon_\text{nominal}$. Using this the truth is folded to give us a reco $r_\text{nominal}$ and data, $N$, is generated by poission varying the reco.

In [3]:
t_nominal = [500, 300] 
M_nominal = np.array([[0.80, 0.20],
                      [0.20, 0.80]])  
a_nominal = [0.75, 0.78]
e_nominal = [0.92, 0.90]
r_nominal = folding(t_nominal, M_nominal, e_nominal, a_nominal)
print("r_nominal = %s" % str(r_nominal))

data = np.random.poisson(r_nominal)
print("data =  %s" % str(data))

r_nominal = [562.66666666666674, 394.87179487179492]
data =  [588 450]


Now we can unfold the data to give us our unfolded distribution using the following equation:

$t^{\text{unfold}}_i = \frac{1}{\epsilon_i}\sum_{j}M_{ij}^{-1}a_j N_j$

In [4]:
t_unfold = unfold(data, M_nominal, a_nominal, e_nominal)
print(" %s" % str(t_unfold))

 [511.95652173913044, 356.66666666666669]


Want to check closure

In [5]:
t_closure_check = unfold(r_nominal, M_nominal, a_nominal, e_nominal)
print(" %s" % str(t_closure_check))

 [500.00000000000006, 300.00000000000006]


We now invent a detector systematic variation. This sample has the same truth distribution but a different reco, $r_{\text{s}1}$. To begin with we will use one with a different migration matrix $M_{\text{s}1}$ but the same acceptance and efficiency.

In [6]:
M_s1 = np.array([[0.85, 0.15],
                 [0.15, 0.85]])  

r_s1 = folding(t_nominal, M_s1, e_nominal, a_nominal)
print("systematic_1 =  %s" % str(r_s1))

systematic_1 =  [575.33333333333337, 382.69230769230768]


The systematic variation in the reco is:
    

In [7]:
print("Systematic variation difference = %s %%" % str(percentage_difference(r_nominal, r_s1)))

Systematic variation difference = [-2.2511848341232157, 3.0844155844155994] %


Now we want to see the systematic variation in the unfolded regime. The $r_{\text{s}1}$ is unfolded with $M_{\text{nominal}}$, $a_{\text{nominal}}$ and $\epsilon_{\text{nominal}}$ then compared to $t_{\text{nominal}}$ to asses the systematic uncertainty.

In [8]:
t_s1_unfolded = unfold(r_s1, M_nominal, a_nominal, e_nominal)
print("Systematic variation difference unfolded = %s %%" % str(percentage_difference(t_nominal, t_s1_unfolded)))

Systematic variation difference unfolded = [-3.4420289855072497, 5.8641975308642031] %


In this case we find that the unfolded systematic uncertainity is larger. We now try a second example where the migration matrix and efficiency is the same as the nominal but the acceptance is different, $a_{\text{s}2}$.

In [9]:
a_s2 = [0.79, 0.83]
r_s2 = folding(t_nominal, M_nominal, e_nominal, a_s2)
t_s2_unfolded = unfold(r_s2, M_nominal, a_nominal, e_nominal)
print("Systematic variation difference at reco = %s %%" % str(percentage_difference(r_nominal, r_s2)))
print("Systematic variation difference unfolded = %s %%" % str(percentage_difference(t_nominal, t_s2_unfolded)))

Systematic variation difference at reco = [5.0632911392405155, 6.0240963855421557] %
Systematic variation difference unfolded = [4.848850548036955, 6.5246640570721297] %


In this case we find that the unfolded systematic uncertainity is larger. We now try a third example where the migration matrix and efficiency is the same as the nominal but the efficiency is different, $e_{\text{s}3}$.

In [10]:
e_s3 = [0.98, 0.92]
r_s3 = folding(t_nominal, M_nominal, e_s3, a_nominal)
t_s3_unfolded = unfold(r_s3, M_nominal, a_nominal, e_nominal)
print("Systematic variation difference at reco = %s %%" % str(percentage_difference(r_nominal, r_s3)))
print("Systematic variation difference unfolded = %s %%" % str(percentage_difference(t_nominal, t_s3_unfolded)))

Systematic variation difference at reco = [-5.9715639810426371, -3.5064935064935119] %
Systematic variation difference unfolded = [-6.5217391304347752, -2.2222222222222663] %


From these three simple tests, the unfolded systematic variation is worse in all scenarios. The greatest difference seems to come from when there is a difference in the migration matrix. A different efficiency and acceptance has a smaller difference between reco and unfolded systematic variations.

#### Singal modelling uncertainty

We now use the same original nominal sample and invent a signal modelling sample with a different truth distribution $t_{\text{sm}1}$, reco $r_{\text{sm}1}$, migration matrix $M_{\text{sm}1}$, acceptance $a_{\text{sm}1}$ and efficiency $\epsilon_{\text{sm}1}$.

In [11]:
t_sm1 = [520, 310]
M_sm1 = np.array([[0.81, 0.19],
                    [0.19, 0.81]])
a_sm1 = [0.74, 0.77]
e_sm1 = [0.92, 0.91]
r_sm1 = folding(t_sm1, M_sm1, e_sm1, a_sm1)
print("r_sm1 = %s" % str(r_sm1))


r_sm1 = [596.08513513513515, 414.80129870129878]


At reco level, the ucnertainty is:

In [12]:
print("Systematic variation difference at reco = %s %%" % str(percentage_difference(r_nominal, r_sm1)))

Systematic variation difference at reco = [-5.9393012680927253, -5.0470821386405875] %


We now have two ways to assess the systematic uncertainty from the signal modelling variation in the unfoled regime:
  1. Unfold $r_{\text{sm}1}$ with the nominal migration matrix, acceptance and efficiency and then compare this to $t_{\text{sm}1}$.
  1. Unfold $r_\text{nominal}$ with the signal modelling migration matrix, acceptance and efficeincy and comepare this to $t_\text{nominal}$.
  
These should be equivalent.

##### Method 1

In [13]:
t_sm1_unfold = unfold(r_sm1, M_nominal, a_nominal, e_nominal)
print("t_sm1_unfold = %s" % str(t_sm1_unfold))
print("Systematic variation difference unfolded = %s %%" % str(percentage_difference(t_sm1, t_sm1_unfold)))

t_sm1_unfold = [530.69217116608411, 313.74674096174101]
Systematic variation difference unfolded = [-2.0561867627084824, -1.2086261166906487] %


##### Method 2

In [14]:
t_nominal_unfold = unfold(r_nominal, M_sm1, a_sm1, e_sm1)
print("t_nominal_unfold = %s" % str(t_nominal_unfold))
print("Systematic variation difference unfolded = %s %%" % str(percentage_difference(t_nominal, t_nominal_unfold)))

t_nominal_unfold = [489.99413816664855, 296.29671238604243]
Systematic variation difference unfolded = [2.0011723666702892, 1.2344292046525234] %


Giving similar answers but with a sign flip per bin. Now to look at a signal modelling uncertainity which only has a different migration matrix $M_{\text{sm}2}$.

In [15]:
t_sm2 = [520, 310]
M_sm2 = np.array([[0.85, 0.15],
                  [0.15, 0.85]])
a_sm2 = a_nominal
e_sm2 = e_nominal
r_sm2 = folding(t_sm2, M_sm2, e_sm2, a_sm2)
print("r_sm2 = %s" % str(r_sm2))
print("Systematic variation difference at reco = %s %%" % str(percentage_difference(r_nominal, r_sm2)))

print("Method 1")
t_sm2_unfold = unfold(r_sm2, M_nominal, a_nominal, e_nominal)
print("t_sm2_unfold = %s" % str(t_sm2_unfold))
print("Systematic variation difference unfolded = %s %%" % str(percentage_difference(t_sm2, t_sm2_unfold)))
print("Method 2")
t_nominal_unfold_sm2 = unfold(r_nominal, M_sm2, a_sm2, e_sm2)
print("t_nominal_unfold_sm2 = %s" % str(t_nominal_unfold_sm2))
print("Systematic variation difference unfolded = %s %%" % str(percentage_difference(t_nominal, t_nominal_unfold_sm2)))


r_sm2 = [597.98666666666668, 396.03846153846155]
Systematic variation difference at reco = [-6.2772511848341113, -0.29545454545453581] %
Method 1
t_sm2_unfold = [538.0615942028985, 291.53703703703701]
Systematic variation difference unfolded = [-3.4733835005574045, 5.9557945041816103] %
Method 2
t_nominal_unfold_sm2 = [485.24844720496907, 315.07936507936518]
Systematic variation difference unfolded = [2.9503105590061867, -5.026455026455058] %


Now to look at a signal modelling uncertainity which only has a different acceptance $a_{\text{sm}3}$.

In [19]:
t_sm3 = [520, 310]
M_sm3 = M_nominal
e_sm3 = e_nominal
a_sm3 = [0.79, 0.83]
r_sm3 = folding(t_sm3, M_sm3, e_sm3, a_sm3)

print("r_sm3 = %s" % str(r_sm3))
print("Systematic variation difference at reco = %s %%" % str(percentage_difference(r_nominal, r_sm3)))

print("Method 1")
t_sm3_unfold = unfold(r_sm3, M_nominal, a_nominal, e_nominal)
print("t_sm3_unfold = %s" % str(t_sm3_unfold))
print("Systematic variation difference unfolded = %s %%" % str(percentage_difference(t_sm3, t_sm3_unfold)))
print("Method 2")
t_nominal_unfold_sm3 = unfold(r_nominal, M_sm3, a_sm3, e_sm3)
print("t_nominal_unfold_sm3 = %s" % str(t_nominal_unfold_sm3))
print("Systematic variation difference unfolded = %s %%" % str(percentage_difference(t_nominal, t_nominal_unfold_sm3)))

r_sm3 = [555.08860759493678, 384.19277108433738]
Systematic variation difference at reco = [1.3468114463975061, 2.7044281020184671] %
Method 1
t_sm3_unfold = [494.78096425327067, 289.76481114330744]
Systematic variation difference unfolded = [4.8498145666787185, 6.5274802763524393] %
Method 2
t_nominal_unfold_sm3 = [525.46488294314383, 320.91396011396006]
Systematic variation difference unfolded = [-5.0929765886287672, -6.9713200379866862] %


Now to look at a signal modelling uncertainity which only has a different efficiency $e_{\text{sm}4}$.

In [20]:
t_sm4 = [520, 310]
M_sm4 = M_nominal
e_sm4 = [0.98, 0.92]
a_sm4 = a_nominal
r_sm4 = folding(t_sm4, M_sm4, e_sm4, a_sm4)

print("r_sm4 = %s" % str(r_sm4))
print("Systematic variation difference at reco = %s %%" % str(percentage_difference(r_nominal, r_sm4)))

print("Method 1")
t_sm4_unfold = unfold(r_sm4, M_nominal, a_nominal, e_nominal)
print("t_sm4_unfold = %s" % str(t_sm4_unfold))
print("Systematic variation difference unfolded = %s %%" % str(percentage_difference(t_sm4, t_sm4_unfold)))
print("Method 2")
t_nominal_unfold_sm4 = unfold(r_nominal, M_sm4, a_sm4, e_sm4)
print("t_nominal_unfold_sm4 = %s" % str(t_nominal_unfold_sm4))
print("Systematic variation difference unfolded = %s %%" % str(percentage_difference(t_nominal, t_nominal_unfold_sm4)))

r_sm4 = [619.62666666666667, 423.17948717948724]
Systematic variation difference at reco = [-10.12322274881515, -7.1688311688311721] %
Method 1
t_sm4_unfold = [553.91304347826076, 316.88888888888891]
Systematic variation difference unfolded = [-6.5217391304347618, -2.2222222222222303] %
Method 2
t_nominal_unfold_sm4 = [469.3877551020409, 293.47826086956525]
Systematic variation difference unfolded = [6.1224489795918196, 2.173913043478251] %


Checking the example in our analysis:

We only have the truth for some samples, for example the aMC@NLO sample, and we are assuming the nominal folding ingredients can be applied to the truth to give us the reco. In this case we are saying for the aMC@NLO sample the turth is different, and therefore the reco also, but the migration, acceptance and efficiency is the same as the nominal.

In [20]:
t_sm5 = [520, 310]
M_sm5 = M_nominal
e_sm5 = e_nominal
a_sm5 = a_nominal
r_sm5 = folding(t_sm5, M_sm5, e_sm5, a_sm5)


print("r_sm5 = %s" % str(r_sm5))
print("Systematic variation difference at reco = %s %%" % str(percentage_difference(r_nominal, r_sm5)))

print("Method 1")
t_sm5_unfold = unfold(r_sm5, M_nominal, a_nominal, e_nominal)
print("t_sm5_unfold = %s" % str(t_sm5_unfold))
print("Systematic variation difference unfolded = %s %%" % str(percentage_difference(t_sm5, t_sm5_unfold)))
print("Method 2")
t_nominal_unfold_sm5 = unfold(r_nominal, M_sm5, a_sm5, e_sm5)
print("t_nominal_unfold_sm5 = %s" % str(t_nominal_unfold_sm5))
print("Systematic variation difference unfolded = %s %%" % str(percentage_difference(t_nominal, t_nominal_unfold_sm5)))

r_sm5 = [584.69333333333338, 408.82051282051282]
Systematic variation difference at reco = [-3.914691943127957, -3.5324675324675194] %
Method 1
t_sm5_unfold = [520.0, 310.0]
Systematic variation difference unfolded = [0.0, 0.0] %
Method 2
t_nominal_unfold_sm5 = [500.00000000000006, 300.00000000000006]
Systematic variation difference unfolded = [-1.1368683772161604e-14, -1.8947806286936007e-14] %


# Stress test
Stress tests are used to check that if the data you unfold has a different signal shape to the samples used to train the migration matrix, the data will still be correctly unfolded to the truth which corresponds to it.

In [24]:
t_stress = [300, 500]
M_stress = np.array([[0.85, 0.15],
                     [0.15, 0.85]])
a_stress = a_nominal
e_stress = e_nominal
r_stress = folding(t_stress, M_stress, e_stress, a_stress)
print("r_stress = %s" % str(r_stress))


r_stress = [402.80000000000001, 543.46153846153845]


In [25]:
t_unfold = unfold(r_stress, M_nominal, a_nominal, e_nominal)
print(t_unfold)

[284.23913043478262, 516.1111111111112]
